In [24]:
from sagemaker import get_execution_role

#Bucket location to save your custom code in tar.gz format.
custom_code_upload_location = 's3://jbs-dataset/customcode'

#Bucket location where results of model training are saved.
model_artifacts_location = 's3://jbs-dataset/artifacts'

#IAM execution role that gives SageMaker access to resources in your AWS account.
role = get_execution_role()

In [25]:
!cat "dnn_classifier.py"

import numpy as np
import os
import tensorflow as tf

INPUT_TENSOR_NAME = 'inputs'

# Disable MKL to get a better perfomance for this model.
os.environ['TF_DISABLE_MKL'] = '1'
os.environ['TF_DISABLE_POOL_ALLOCATOR'] = '1'


def estimator_fn(run_config, params):
    feature_columns = [tf.feature_column.numeric_column(INPUT_TENSOR_NAME, shape=[29])]
    return tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                      hidden_units=[10, 20, 10],
                                      n_classes=2,
                                      config=run_config)


def serving_input_fn(params):
    feature_spec = {INPUT_TENSOR_NAME: tf.FixedLenFeature(dtype=tf.float32, shape=[29])}
    return tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)()


def train_input_fn(training_dir, params):
    """Returns input function that would feed the model during training"""
    return _generate_input_fn(training_dir, 'train_data.csv')


def eval_input_

In [26]:
def estimator(model_path, hyperparameters):
    feature_columns = [tf.feature_column.numeric_column(INPUT_TENSOR_NAME, shape=[29])]
    return tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                      hidden_units=[10, 20, 10],
                                      n_classes=2,
                                      model_dir=model_path)

In [27]:
def train_input_fn(training_dir, hyperparameters):
    training_set = tf.contrib.learn.datasets.base.load_csv_without_header(
        filename=os.path.join(training_dir, 'train.csv'),
        target_dtype=np.int,
        features_dtype=np.float32)

    return tf.estimator.inputs.numpy_input_fn(
        x={INPUT_TENSOR_NAME: np.array(training_set.data)},
        y=np.array(training_set.target),
        num_epochs=None,
        shuffle=True)()

In [28]:
def serving_input_fn(hyperparameters):
    feature_spec = {INPUT_TENSOR_NAME: tf.FixedLenFeature(dtype=tf.float32, shape=[29])}
    return tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)()

In [29]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(entry_point='dnn_classifier.py',
                            role=role,
                            framework_version='1.9',
                            output_path=model_artifacts_location,
                            code_location=custom_code_upload_location,
                            train_instance_count=1,
                            train_instance_type='ml.c4.xlarge',
                            training_steps=1000,
                            evaluation_steps=100)

In [30]:
%%time
import boto3

# use the region-specific sample data bucket
region = boto3.Session().region_name
train_data_location = 's3://jbs-dataset'.format(region)

estimator.fit(train_data_location)

2021-02-16 00:51:42 Starting - Starting the training job...
2021-02-16 00:51:44 Starting - Launching requested ML instances......
2021-02-16 00:52:50 Starting - Preparing the instances for training......
2021-02-16 00:54:11 Downloading - Downloading input data
2021-02-16 00:54:11 Training - Downloading the training image...
2021-02-16 00:54:26 Training - Training image download completed. Training in progress.2021-02-16 00:54:26,664 INFO - root - running container entrypoint
2021-02-16 00:54:26,664 INFO - root - starting train task
2021-02-16 00:54:26,670 INFO - container_support.training - Training starting
2021-02-16 00:54:30,434 INFO - tf_container - ----------------------TF_CONFIG--------------------------
2021-02-16 00:54:30,434 INFO - tf_container - {"environment": "cloud", "cluster": {"master": ["algo-1:2222"]}, "task": {"index": 0, "type": "master"}}
2021-02-16 00:54:30,434 INFO - tf_container - ---------------------------------------------------------
2021-02-16 00:54:30,434 I

### Deploy the trained Model  

In [31]:
%%time
predictor = estimator.deploy(initial_instance_count=1,
                                       instance_type='ml.m4.xlarge')

-------------!CPU times: user 331 ms, sys: 17.9 ms, total: 348 ms
Wall time: 6min 32s


###  Invoke the Endpoint to get inferences

In [32]:
predictor.predict([-15.819178720771802,8.7759971528627,-22.8046864614815,11.864868080360699,-9.09236053189517,-2.38689320657655,-16.5603681078199,0.9483485947860579,-6.31065843275059,-13.0888909176936,9.81570317447819,-14.0560611837648,0.777191846436601,-13.7610179615936,-0.353635939812489,-7.9574472262599505,-11.9629542349435,-4.7805077876172,0.652498045264831,0.992278949261366,-2.35063374523783,1.03636187430048,1.13605073696052,-1.0434137405139001,-0.10892334328197999,0.657436778462222,2.1364244708551396,-1.41194537483904,-0.3492313067728856]) #expected label to be 1

{'model_spec': {'name': u'generic_model',
  'signature_name': u'serving_default',
  'version': {'value': 1613436887L}},
 'result': {'classifications': [{'classes': [{'label': u'0',
      'score': 0.026686491444706917},
     {'label': u'1', 'score': 0.973313570022583}]}]}}

### (Optional) Delete the Endpoint

In [ ]:
print(predictor.endpoint)

In [ ]:
import sagemaker

sagemaker.Session().delete_endpoint(predictor.endpoint)